In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np
import statsmodels.api as sm
import folium

In [2]:
df = pd.read_csv('./data/kc_house_data.csv', parse_dates = ['date'])

house = df.copy()

In [3]:
house.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [4]:
house = house.drop(['id', 'date', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15'], axis = 1)

In [5]:
house.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,zipcode,lat,long
0,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,Average,7 Average,1180,0.0,98178,47.5112,-122.257
1,538000.0,3,2.25,2570,7242,2.0,NO,NONE,Average,7 Average,2170,400.0,98125,47.7210,-122.319
2,180000.0,2,1.00,770,10000,1.0,NO,NONE,Average,6 Low Average,770,0.0,98028,47.7379,-122.233
3,604000.0,4,3.00,1960,5000,1.0,NO,NONE,Very Good,7 Average,1050,910.0,98136,47.5208,-122.393
4,510000.0,3,2.00,1680,8080,1.0,NO,NONE,Average,8 Good,1680,0.0,98074,47.6168,-122.045


In [6]:
abnb_df = pd.read_csv('data/listings (2).csv')

In [7]:
abnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4686 entries, 0 to 4685
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              4686 non-null   int64  
 1   name                            4686 non-null   object 
 2   host_id                         4686 non-null   int64  
 3   host_name                       4686 non-null   object 
 4   neighbourhood_group             4686 non-null   object 
 5   neighbourhood                   4686 non-null   object 
 6   latitude                        4686 non-null   float64
 7   longitude                       4686 non-null   float64
 8   room_type                       4686 non-null   object 
 9   price                           4686 non-null   int64  
 10  minimum_nights                  4686 non-null   int64  
 11  number_of_reviews               4686 non-null   int64  
 12  last_review                     39

In [8]:
abnb_cleaned = abnb_df.drop(['id', 'name', 'host_id', 'host_name', 'last_review', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license'], axis=1)

abnb_cleaned.head()


,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month
0,Central Area,Madrona,47.61094,-122.29286,Entire home/apt,333,7,36,0.22
1,Other neighborhoods,Wallingford,47.65444,-122.33629,Entire home/apt,99,3,157,1.04
2,Other neighborhoods,Georgetown,47.55017,-122.31937,Private room,62,2,154,1.11
3,West Seattle,Fairmount Park,47.55495,-122.38663,Entire home/apt,185,2,54,0.45
4,West Seattle,Fairmount Park,47.55627,-122.38607,Entire home/apt,145,5,59,0.49


In [9]:

seattle_lat_max = abnb_cleaned['latitude'].max()
seattle_lat_min = abnb_cleaned['latitude'].min()
seattle_lon_max = abnb_cleaned['longitude'].max()
seattle_lon_min = abnb_cleaned['longitude'].min()

house_seattle = house.loc[((house['lat'] >= seattle_lat_min) & (house['lat'] <= seattle_lat_max)) & ((house['long'] >= seattle_lon_min) & (house['long'] <= seattle_lon_max))]
house_seattle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7034 entries, 0 to 21596
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          7034 non-null   float64
 1   bedrooms       7034 non-null   int64  
 2   bathrooms      7034 non-null   float64
 3   sqft_living    7034 non-null   int64  
 4   sqft_lot       7034 non-null   int64  
 5   floors         7034 non-null   float64
 6   waterfront     6277 non-null   object 
 7   view           7011 non-null   object 
 8   condition      7034 non-null   object 
 9   grade          7034 non-null   object 
 10  sqft_above     7034 non-null   int64  
 11  sqft_basement  7034 non-null   object 
 12  zipcode        7034 non-null   int64  
 13  lat            7034 non-null   float64
 14  long           7034 non-null   float64
dtypes: float64(5), int64(5), object(5)
memory usage: 879.2+ KB


In [13]:
# Keeping only necessary columns from the house_seattle dataframe
# house_seattle_locations = house_seattle[['lat', 'long', 'price']]

In [16]:
house_seattle['lat'] = house_seattle['lat'].astype(str)
house_seattle['long'] = house_seattle['long'].astype(str)


<ipython-input-16-624f20d3dcfc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  house_seattle['lat'] = house_seattle['lat'].astype(str)
<ipython-input-16-624f20d3dcfc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  house_seattle['long'] = house_seattle['long'].astype(str)


In [17]:
house_seattle['lat-lon'] = house_seattle[['lat', 'long']].agg('-'.join, axis=1)

<ipython-input-17-8282daf54a7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  house_seattle['lat-lon'] = house_seattle[['lat', 'long']].agg('-'.join, axis=1)


In [19]:
abnb_cleaned['latitude'] = abnb_cleaned['latitude'].astype(str)
abnb_cleaned['longitude'] = abnb_cleaned['longitude'].astype(str)


In [20]:
abnb_cleaned['lat-lon'] = abnb_cleaned[['latitude', 'longitude']].agg('-'.join, axis=1)

In [27]:
abnb_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4686 entries, 0 to 4685
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   neighbourhood_group  4686 non-null   object 
 1   neighbourhood        4686 non-null   object 
 2   latitude             4686 non-null   object 
 3   longitude            4686 non-null   object 
 4   room_type            4686 non-null   object 
 5   price                4686 non-null   int64  
 6   minimum_nights       4686 non-null   int64  
 7   number_of_reviews    4686 non-null   int64  
 8   reviews_per_month    3964 non-null   float64
 9   lat-lon              4686 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 366.2+ KB


In [26]:
# merge house_seattle_locations and abnb_cleaned

total_locations = house_seattle.merge(abnb_cleaned, how='left', on='lat-lon')
total_locations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7034 entries, 0 to 7033
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   price_x              7034 non-null   float64
 1   bedrooms             7034 non-null   int64  
 2   bathrooms            7034 non-null   float64
 3   sqft_living          7034 non-null   int64  
 4   sqft_lot             7034 non-null   int64  
 5   floors               7034 non-null   float64
 6   waterfront           6277 non-null   object 
 7   view                 7011 non-null   object 
 8   condition            7034 non-null   object 
 9   grade                7034 non-null   object 
 10  sqft_above           7034 non-null   int64  
 11  sqft_basement        7034 non-null   object 
 12  zipcode              7034 non-null   int64  
 13  lat                  7034 non-null   object 
 14  long                 7034 non-null   object 
 15  lat-lon              7034 non-null   o

In [21]:
# Creating the map and adding seattle house points to it
map_seattle = folium.Map(location=[house_seattle_locations.lat.mean(), house_seattle_locations.long.mean()], zoom_start=10, control_scale=True)

for index, location_info in house_seattle_locations.iterrows():
    folium.Marker([location_info['lat'], location_info['long']], popup=location_info["price"]).add_to(map_seattle)

In [20]:
map_seattle.save("seattle_houses.html")

In [22]:
# Creating the map and adding seattle airbnb points to it
map_seattle2 = folium.Map(location=[abnb_cleaned.latitude.mean(), abnb_cleaned.longitude.mean()], zoom_start=10, control_scale=True)

for index, location_info in abnb_cleaned.iterrows():
    folium.Marker([location_info['latitude'], location_info['longitude']], popup=location_info["price"]).add_to(map_seattle2)

In [23]:
map_seattle2.save("seattle_abnb.html")